In [1]:
import ee
from ipygee import *
import eeconvert
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
ee.Initialize()

In [3]:
import os,glob
g = os.getcwd()
g
path = 'D:\\repos\\CIAT\\IDB_PROJECT'
os.chdir(path)

In [4]:
def clouds_shadows_mask_L7(image):
    qa = image.select('pixel_qa')
#   If the cloud bit (5) is set and the cloud confidence (7) is high
#   or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = (qa.bitwiseAnd(1 << 5)
             .And(qa.bitwiseAnd(1 << 7))
             .Or(qa.bitwiseAnd(1 << 3)))
#  emove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    mask = image.updateMask(cloud.Not()).updateMask(mask2)
    return mask


def clouds_shadows_mask_L8(image):
#   // Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3);
    cloudsBitMask = (1 << 5);
#   // Get the pixel QA band.
    qa = image.select('pixel_qa');
#   // Both flags should be set to zero, indicating clear conditions.
    mask2 = (qa.bitwiseAnd(cloudShadowBitMask).eq(0)
             .And(qa.bitwiseAnd(cloudsBitMask).eq(0)))
    mask = image.updateMask(mask2) 
    return mask

def reflectance(image):
    return ee.Image(image.multiply(0.0001).copyProperties(image,["system:time_start"]))

bandNames = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2']

def VI_L7(image):
    IVs = (image.addBands(image.normalizedDifference(['NIR', 'RED']).rename('NDVI')))
#     IVs = IVs.addBands(image.normalizedDifference(['B4', 'B5']).rename('NDRE'))
    IVs = (IVs.addBands(image.normalizedDifference(['NIR', 'B2']).rename('GNDVI')))
    IVs = (IVs.addBands(image.normalizedDifference(['B2', 'B5']).rename('DNWVI')))
    IVs = (IVs.addBands(image.normalizedDifference(['B5', 'NIR']).rename('NWVI')))
    IVs = (IVs.addBands(image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': image.select('B4'),
      'RED': image.select('RED'),
      'BLUE': image.select('B1')}).rename('EVI')))
#     IVs = (IVs.addBands(image.expression(
#     '(NIR * (1 - RED) * (NIR - RED))^(1/3)', {
#       'NIR': image.select('B4'),
#       'RED': image.select('B3')}).rename('AVI')))
    IVs = (IVs.addBands(image.expression(
    '(NIR - 2 * RED + BLUE)/(NIR + 2 * RED - BLUE)', {
      'NIR': image.select('B4'),
      'RED': image.select('RED'),
      'BLUE': image.select('B1')}).rename('ARVI')))
    IVs =( IVs.addBands(image.expression(
    'NIR/(NIR + RED)', {
      'NIR': image.select('B4'),
      'RED': image.select('RED'),}).rename('IPVI')))
    IVs = (IVs.addBands(image.expression(
    '(NIR-RED)/(NIR+RED + 0.16)', {
      'NIR': image.select('B4'),
      'RED': image.select('B3'),}).rename('OSAVI')))
    IVs = (IVs.addBands(image.expression(
    'GREEN - 0.39 * RED - 0.69 * BLUE', {
      'BLUE': image.select('B1'),
      'RED': image.select('B3'),
      'GREEN': image.select('B2'),}).rename('TGI')))
    return IVs

def VI_L8(image):
    IVs = (image.addBands(image.normalizedDifference(['B5', 'B4']).rename('NDVI')))
#     IVs = IVs.addBands(image.normalizedDifference(['B4', 'B5']).rename('NDRE'))
    IVs = (IVs.addBands(image.normalizedDifference(['B5', 'B2']).rename('GNDVI')))
    IVs = (IVs.addBands(image.normalizedDifference(['B2', 'B6']).rename('DNWVI')))
    IVs = (IVs.addBands(image.normalizedDifference(['B6', 'B5']).rename('NWVI')))
    IVs = (IVs.addBands(image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': image.select('B5'),
      'RED': image.select('B4'),
      'BLUE': image.select('B2')}).rename('EVI')))
#     IVs = (IVs.addBands(image.expression(
#     '(NIR * (1 - RED) * (NIR - RED))^(1/3)', {
#       'NIR': image.select('B5'),
#       'RED': image.select('B4')}).rename('AVI')))
    IVs = (IVs.addBands(image.expression(
    '(NIR - 2 * RED + BLUE)/(NIR + 2 * RED - BLUE)', {
      'NIR': image.select('B5'),
      'RED': image.select('B4'),
      'BLUE': image.select('B2')}).rename('ARVI')))
    IVs =( IVs.addBands(image.expression(
    'NIR/(NIR + RED)', {
      'NIR': image.select('B5'),
      'RED': image.select('B4'),}).rename('IPVI')))
    IVs = (IVs.addBands(image.expression(
    '(NIR-RED)/(NIR+RED + 0.16)', {
      'NIR': image.select('B5'),
      'RED': image.select('B4'),}).rename('OSAVI')))
    IVs = (IVs.addBands(image.expression(
    'GREEN - 0.39 * RED - 0.69 * BLUE', {
      'BLUE': image.select('B2'),
      'RED': image.select('B4'),
      'GREEN': image.select('B3'),}).rename('TGI')))
    return IVs

In [5]:
def preprocessing(imageCollection,Cloud_masking = True, calculateReflectance = True, CalculateVIs = True, Landsat = 8):
    if Cloud_masking:
        if Landsat == 8:
            imageCollection = imageCollection.map(clouds_shadows_mask_L8)
        if Landsat == 7:
            imageCollection = imageCollection.map(clouds_shadows_mask_L7)
    if calculateReflectance:
        imageCollection = imageCollection.map(reflectance)
    if CalculateVIs:
        if Landsat == 8:
            imageCollection = imageCollection.map(VI_L8)
        if Landsat == 7:
            imageCollection = imageCollection.map(VI_L7)
    return imageCollection

# covert shp to FeatureCollection

In [8]:
Points = gpd.read_file("Data_IDB/new/Zona_influencia_DR.shp")
# print(Points.crs)
Points = Points.to_crs("EPSG:4326")
ID = str(Points["cuest_id_b"]) + str(Points["parcela"])
str1 = [str(int) for int in Points["cuest_id_b"]] 
str2 = [str(int) for int in Points["parcela"]]
Points["ID"] = [s1 + s2 for s1, s2 in zip(str1, str2)]
ok = Points[Points['corregido2'].str.contains("ok")]
Points = ok[["ID","patca",'geometry']]
# Points.to_csv("test.csv")
Points = eeconvert.gdfToFc(Points)
Points

# FUNCTIONS TEST

In [ ]:
L7 = (ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
      .filterBounds(Points)
#       .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',35))# filter according to drawn boundary
      .filterDate('2011-01-01', '2011-01-30'))
#       .map(lambda image: image.clipToBoundsAndScale(points)))
pos_L7 = preprocessing(L7,Cloud_masking = True, calculateReflectance = True, CalculateVIs = True, Landsat = 7)

In [ ]:
Img_list = pos_L7.toList(pos_L7.size())
# Img_list.get(0).getInfo()
alldf = pd.DataFrame()
for i in range(0,Img_list.size().getInfo()):
    print(i)
    img = ee.Image(Img_list.get(i)).select(['NDVI','GNDVI','DNWVI','NWVI','EVI','ARVI','IPVI','OSAVI','TGI'])
    test = img.reduceRegions(Points,ee.Reducer.mean(),30)
    test = eeconvert.fcToGdf(test,crs={'init': 'epsg:4326'})
    test.dropna(inplace = True)
    date = img.date().format().getInfo()
    test["date"] = date.split("T")[0]
    test['date'] = pd.to_datetime(test['date'], format='%Y-%m-%d')
    alldf = pd.concat([alldf,test])
def extractFeatures(img, FeatureCollection,Bands = ["NDVI","NDRE"]):
    img = img.select(Bands)
    FT = img.reduceRegions(FeatureCollection,ee.Reducer.mean(),30)
    FT = eeconvert.fcToGdf(test,crs={'init': 'epsg:4326'})
    FT.dropna(inplace = True)
    date = img.date().format().getInfo()
    FT["date"] = date.split("T")[0]
    FT['date'] = pd.to_datetime(FT['date'], format='%Y-%m-%d')
    return FT

# run bash

In [9]:
y = range(2011,2020)
for year in y:
    start = ee.Date.fromYMD(year,1,1)
    end = ee.Date.fromYMD(year,12,31)
    dataset7_T1 = (ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
                   .filterBounds(Points)
#                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',35))# filter according to drawn boundary
                   .filterDate(start,end))####T1 is Data that meets geometric and radiometric quality requirements
    dataset8_T1 = (ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
                   .filterBounds(Points)
#                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',35))# filter according to drawn boundary
                   .filterDate(start,end))
    Len_L7 = dataset7_T1.size().getInfo()
    Len_L8 = dataset8_T1.size().getInfo()
    print("L7: ",Len_L7)
    print("L8: ",Len_L8)
    if Len_L7 != 0:
        pos_L7 = preprocessing(dataset7_T1,Cloud_masking = True, calculateReflectance = True, CalculateVIs = True, Landsat = 7)
        Img_list = pos_L7.toList(pos_L7.size())
        alldf7 = pd.DataFrame()
        for i in range(0,Len_L7):
            img = ee.Image(Img_list.get(i)).select(['NDVI','GNDVI','DNWVI','NWVI','EVI','ARVI','IPVI','OSAVI','TGI'])
            test = img.reduceRegions(Points,ee.Reducer.mean(),30)
            test = eeconvert.fcToGdf(test,crs={'init': 'epsg:4326'})
            test.dropna(inplace = True)
            date = img.date().format().getInfo()
            test["date"] = date.split("T")[0]
            test['date'] = pd.to_datetime(test['date'], format='%Y-%m-%d')
            alldf7 = pd.concat([alldf7,test])
        alldf7.drop('geometry', inplace=True, axis=1)
        alldf7.to_csv("Data_IDB/FT/L7_"+str(year)+".csv",index=False)
        print("L7_"+str(year), "DONE")
    else:
        print("L7_"+str(year), ": Not found images")
    if Len_L8 != 0:
        pos_L8 = preprocessing(dataset8_T1,Cloud_masking = True, calculateReflectance = True, CalculateVIs = True, Landsat = 8)
        Img_list = pos_L8.toList(pos_L8.size())
        alldf8 = pd.DataFrame()
        for i in range(0,Len_L8):
            img = ee.Image(Img_list.get(i)).select(['NDVI','GNDVI','DNWVI','NWVI','EVI','ARVI','IPVI','OSAVI','TGI'])
            test = img.reduceRegions(Points,ee.Reducer.mean(),30)
            test = eeconvert.fcToGdf(test,crs={'init': 'epsg:4326'})
            test.dropna(inplace = True)
            date = img.date().format().getInfo()
            test["date"] = date.split("T")[0]
            test['date'] = pd.to_datetime(test['date'], format='%Y-%m-%d')
            alldf8 = pd.concat([alldf8,test])
        alldf8.drop('geometry', inplace=True, axis=1)
        alldf8.to_csv("Data_IDB/FT/L8_"+str(year)+".csv",index=False)
        print("L8_"+str(year),": DONE")
    else:
        print("L8_"+str(year), ": Not found images")

L7:  52
L8:  0


C:\Users\fmontenegro\Miniconda3\envs\GEE\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


L7_2011 DONE
L8_2011 : Not found images
L7:  58
L8:  0
L7_2012 DONE
L8_2012 : Not found images
L7:  50
L8:  58
L7_2013 DONE
L8_2013 : DONE
L7:  2
L8:  103
L7_2014 DONE
L8_2014 : DONE
L7:  0
L8:  103
L7_2015 : Not found images
L8_2015 : DONE
L7:  7
L8:  99
L7_2016 DONE
L8_2016 : DONE
L7:  0
L8:  95
L7_2017 : Not found images
L8_2017 : DONE
L7:  0
L8:  107
L7_2018 : Not found images
L8_2018 : DONE
L7:  0
L8:  107
L7_2019 : Not found images
L8_2019 : DONE
